<a href="https://colab.research.google.com/github/eshaanrathi2/vada-pav-classifier/blob/master/vada_pav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import keras
from keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input #importing preprocessing unit for inceptionv3 has a different syntax
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint # to save our model

Using TensorFlow backend.


In [0]:
datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = datagen.flow_from_directory('/content/gdrive/My Drive/vada_pav/dataset/train', # this is where you specify the path to the main data folder
                                                 target_size = (224,224),
                                                 color_mode = 'rgb',
                                                 batch_size = 32,
                                                 shuffle = True)

Found 658 images belonging to 2 classes.


In [5]:
test_generator = datagen.flow_from_directory('/content/gdrive/My Drive/vada_pav/dataset/test', # this is where you specify the path to the main data folder
                                                 target_size = (224,224),
                                                 color_mode = 'rgb',
                                                 batch_size = 32,
                                                 shuffle = True)

Found 159 images belonging to 2 classes.


Architechture Construction : 

In [6]:
base_model = InceptionV3(weights = 'imagenet',include_top = False, input_shape = (224,224,3))
x = Flatten()(base_model.output)
x = Dense(2, activation = 'softmax')(x)

model = Model(base_model.input, x)














87916544/87910968 [==============================] - 2s 0us/step


In [0]:
for layer in base_model.layers:
    layer.trainable = False


In [8]:
model.compile(optimizer = Adam(lr=0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
checkpointer = ModelCheckpoint(filepath = '/content/gdrive/My Drive/vada_pav/weights.hdf5', verbose=1, save_best_only=True)

In [0]:
step_size_train = train_generator.n//train_generator.batch_size

Few images are corrupted. Hence following lines would solve the issue. Otherwise, a "image file is truncated" error will occur.

In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Training : 

In [16]:
history = model.fit(train_generator, epochs=15, callbacks = [checkpointer], steps_per_epoch = step_size_train)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/15
12/20 [=================>............] - ETA: 1:21 - loss: 1.7430 - acc: 0.6872

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


20/20 [==============================] - 199s 10s/step - loss: 1.3396 - acc: 0.7514
Epoch 2/15


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


20/20 [==============================] - 16s 817ms/step - loss: 0.2164 - acc: 0.9351
Epoch 3/15
20/20 [==============================] - 6s 290ms/step - loss: 0.1758 - acc: 0.9546
Epoch 4/15
20/20 [==============================] - 6s 323ms/step - loss: 0.0958 - acc: 0.9687
Epoch 5/15
20/20 [==============================] - 6s 288ms/step - loss: 0.1840 - acc: 0.9457
Epoch 6/15
20/20 [==============================] - 6s 301ms/step - loss: 0.4733 - acc: 0.8875
Epoch 7/15
20/20 [==============================] - 5s 254ms/step - loss: 0.3360 - acc: 0.9511
Epoch 8/15
20/20 [==============================] - 7s 329ms/step - loss: 0.0634 - acc: 0.9844
Epoch 9/15
20/20 [==============================] - 6s 320ms/step - loss: 0.0396 - acc: 0.9891
Epoch 10/15
20/20 [==============================] - 5s 247ms/step - loss: 0.0265 - acc: 0.9879
Epoch 11/15
20/20 [==============================] - 7s 331ms/step - loss: 0.0347 - acc: 0.9922
Epoch 12/15
20/20 [==============================] - 6s 28

Evaluation (on unseen data) :

In [17]:
step_size_test = test_generator.n//test_generator.batch_size
model.evaluate_generator(test_generator, verbose=1, steps=step_size_test)

4/4 [==============================] - 38s 10s/step


[0.8926222249865532, 0.890625]

Predictions : 

In [19]:
test_generator.reset()
preds = model.predict_generator(test_generator, steps= step_size_test, verbose=1)

4/4 [==============================] - 1s 241ms/step


In [0]:
predicted_class_indices = np.argmax(preds,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [28]:
print(predictions[1], labels[1], predicted_class_indices[1] )

vada_pav vada_pav 1


In [0]:
model.save_weights('/content/gdrive/My Drive/vada_pav/weights.hdf5')